In [26]:
conn = sqlite3.connect('C:/Users/huber/OneDrive/NHL/DbMatchs.db')

#fonction qui crée une df du game log d'un gardien
def logGardien(idNHL):
    jsonJoueur = requests.get("https://statsapi.web.nhl.com/api/v1/people/" + idNHL + "/stats?stats=gameLog&season=20202021").json()
    df = pd.json_normalize(jsonJoueur["stats"][0]["splits"])
    gardienLogs = df[["season", "date", "isWin", "stat.shutouts"]]
    gardienLogs["gardien"] = idNHL
    return gardienLogs

#générer la liste de tous les gardiens de la table JOUEURS 
dfGardiens = pd.read_sql("SELECT idNHL FROM JOUEURS WHERE position == 'G'", conn)
gardiens = dfGardiens["idNHL"].apply(str)
listGardiens = gardiens.values.tolist()

#créer une df vide pour plus tard append le return de logGardien(idNHL) 
vraisCol = ["season", "date", "isWin", "stat.shutouts", "gardien"]
gameLogsG = pd.DataFrame(columns=vraisCol)
gameLogsG = gameLogsG.fillna(0)

#itérer logGardien(idNHL) à la liste des gardiens de la table JOUEURS et sauvegarder les résultats à gameLogsG
for i in listGardiens:
    try:
        gardienLogs = logGardien(i)
    except:
        print("fausse alarme (espoir) " + i)
    gameLogsG = gameLogsG.append(gardienLogs)
gameLogsG = gameLogsG.rename(columns = {"date" : "dateMatch"})
gameLogsG["dateMatch"] = pd.to_datetime(gameLogsG["dateMatch"])
gameLogsG.dtypes

fausse alarme (espoir) 8482137
fausse alarme (espoir) 8481519


season                   object
dateMatch        datetime64[ns]
isWin                    object
stat.shutouts            object
gardien                  object
dtype: object

In [28]:
# save la df des game logs en une table SQL
gameLogsG.to_sql("GARDIENS", conn, if_exists="replace", index=False, dtype={"dateMatch": sqlalchemy.DateTime()})
# faire le update SQL

ValueError: dateMatch (DATETIME) not a string